<a href="https://colab.research.google.com/github/BamByeol/learning-ML/blob/HW1120/HW1120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#사용할 library & Data Import
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#사용할 데이터 셋 (Boston Housing Dataset)
df = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/BostonHousing2.csv")
df.head()

In [ ]:
#2.데이터 파악(EDA : 탐색적 데이터 분석)
#그래프 배경 설정
sns.set_style('darkgrid')

#2-1 데이터 셋 기본 정보 파악

#데이터 셋의 기본정보
df.shape
#결측치
df.isnull().sum()
#각 변수의 타입 및 기초 통계량 확인
df.info()
#numerical variable
df.describe()
#categorical variable
num_town = df['TOWN'].unique()
print(len(num_town))
num_town

In [ ]:
#2-2 종속변수(목표변수) 탐색
#기초 통계량
df['CMEDV'].describe()
#분포
df['CMEDV'].hist(bins=50)


In [ ]:
#boxplot - Pandas
df.boxplot(column=['CMEDV'])
plt.show()

In [ ]:
#boxlot - matplotlib
plt.boxplot(df['CMEDV'])
plt.show()

In [ ]:
#2-3 설명변수 탐색
#설명변수의 분포 탐색
# numerical features (except "LON" & "LAT")
numerical_columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

fig = plt.figure(figsize = (16, 20))
ax = fig.gca()  # Axes 생성

df[numerical_columns].hist(ax=ax)
plt.show()

In [ ]:
#2-4 설명변수와 종속변수 간의 관계탐색
#변수 간의 상관계수 파악
# Person 상관계수
cols = ['CMEDV', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

corr = df[cols].corr(method = 'pearson')
corr

In [ ]:
# 상관계수를 좀 더 직관적인 heatmap으로 표현
# heatmap (seaborn)
fig = plt.figure(figsize = (16, 12))
ax = fig.gca()

sns.set(font_scale = 1.5)  # heatmap 안의 font-size 설정
heatmap = sns.heatmap(corr.values, annot = True, fmt='.2f', annot_kws={'size':15},
                      yticklabels = cols, xticklabels = cols, ax=ax, cmap = "RdYlBu")
plt.tight_layout()
plt.show()

In [ ]:
#종속변수와 설명 변수간의 관계 탐색
#주택가격('CMEDV') ~ 방 갯수 ("RM")
# scatter plot
sns.scatterplot(data=df, x='RM', y='CMEDV', markers='o', color='blue', alpha=0.6)
plt.title('Scatter Plot')
plt.show()

In [ ]:
#주택가격("CMEDV") ~ 빈곤층의 비율("LSTAT")
# scatter plot
sns.scatterplot(data=df, x='LSTAT', y='CMEDV', markers='o', color='blue', alpha=0.6)
plt.title('Scatter Plot')
plt.show()

In [ ]:
#도시별 차이 탐색

#도시별 데이터 갯수
df['TOWN'].value_counts()

#도시별 데이터 갯수 (bar plot)
df['TOWN'].value_counts().hist(bins=50)

#도시별 주택 가격 특징 (boxplot 이용)
fig = plt.figure(figsize = (12, 20))
sns.boxplot(x='CMEDV', y='TOWN', data=df)

#도시별 범죄율 특징
fig = plt.figure(figsize = (12, 20))
sns.boxplot(x='CRIM', y='TOWN', data=df)

In [ ]:
# 3.주택가격 에측 모델링 : 회귀분석
#데이터 전처리
#Feature 표준화 (scale 차이를 없애기 위해 수치형 Feature에 대해서 표준화를 진행)
df.head()
df.info()


In [ ]:
# 문자형 변수인 'TOWN'와 범주형 변수인 'CHAS'(Dummy variable)를 제와하여 모든 수치형 변수에 대해 표준화를 진행
from sklearn.preprocessing import StandardScaler

# feature standardization  (numerical_columns except dummy var.-"CHAS")
scaler = StandardScaler()  # 평균 0, 표준편차 1
scale_columns = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
df[scale_columns] = scaler.fit_transform(df[scale_columns])

df.head()

In [ ]:
#training set / test set 나누기
# features for linear regression model
df[numerical_columns].head()

from sklearn.model_selection import train_test_split

# split dataset into training & test
X = df[numerical_columns]
y = df['CMEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#X_train.shape, Y_train.shape
#((404, 13)), (404,))

#X_test.shape, Y_test,shape
#((102, 13)), (102,))

In [ ]:
#다중공선성의 존재여부 확인
# 보통 VIF 값을 확인, VIF >10인 feature 들은 다른 변수와의 상관관계가 높아 다중공선성이 존재하는 것으로 판단
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['features'] = X_train.columns
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif.round(1)

In [ ]:
#3-2 회귀 모델링
#training set에서 선형회귀예측모델을 학습, 도출될 모델을 test set에 적용행 주택가격 ('CMEDV')를 에측

from sklearn import linear_model

# fit regression model in training set
lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)

# predict in test set
pred_test = lr.predict(X_test)

In [ ]:
#3-3 모델 해석

#각 feature의 회귀 계수확인
# print coef
print(lr.coef_)

# "feature - coefficients" DataFrame 만들기
coefs = pd.DataFrame(zip(df[numerical_columns].columns, lr.coef_), columns = ['feature', 'coefficients'])
coefs

#크기 순서로 나열
coefs_new = coefs.reindex(coefs.coefficients.abs().sort_values(ascending=False).index)
coefs_new

In [ ]:
## coefficients 시각화

# figure size
plt.figure(figsize = (8, 8))

# bar plot
plt.barh(coefs_new['feature'], coefs_new['coefficients'])
plt.title('"feature - coefficient" Graph')
plt.xlabel('coefficients')
plt.ylabel('features')
plt.show()

In [ ]:
#feature 유의성 검정
import statsmodels.api as sm

X_train2 = sm.add_constant(X_train)
model2 = sm.OLS(y_train, X_train2).fit()
model2.summary()

In [ ]:
#3-4 모델 예측 결과 및 성능 평가

# 예측 결과 시각화 (test set)
df = pd.DataFrame({'actual': y_test, 'prediction': pred_test})
df = df.sort_values(by='actual').reset_index(drop=True)

plt.figure(figsize=(12, 9))
plt.scatter(df.index, df['prediction'], marker='x', color='r')
plt.scatter(df.index, df['actual'], alpha=0.7, marker='o', color='black')
plt.title("Prediction Result in Test Set", fontsize=20)
plt.legend(['prediction', 'actual'], fontsize=12)
plt.show()

In [ ]:
#모델 성능 평가

# R square
print(model.score(X_train, y_train))  # training set
print(model.score(X_test, y_test))  # test set

print('-----------------------------------------------------')


# RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt

# training set
pred_train = lr.predict(X_train)
print(sqrt(mean_squared_error(y_train, pred_train)))

# test set
print(sqrt(mean_squared_error(y_test, pred_test)))

# all rigth reserved (https://hyemin-kim.github.io/2020/08/11/E-Python-LinearRegression-1/)